In [2]:
import pandas as pd
import numpy as np
# import torch
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertModel, BertTokenizer
import csv


/Users/yoonji_kim/delab/tuning/parsing_mysql_parameter/myvenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# CSV 파일 불러오기
# mysql_csv = pd.read_csv('mysql_parameters.csv')
# mysql_csv = pd.read_csv('mysql_parameters_using_lasso_new.csv')
mysql_csv = pd.read_csv('../csv/mysql_parameters_using_lasso.csv')
postgresql_csv = pd.read_csv('../csv/postgre_parameters.csv')

In [17]:
mysql_csv.head(1)

,Parameter,Description
0,innodb-buffer-pool-size=#,The size of the memory buffer InnoDB uses to c...


In [18]:
mysql_csv['Description'] = mysql_csv['Description'].apply(lambda x:  x.replace("InnoDB","server"))

In [19]:
mysql_csv[mysql_csv.Parameter == 'innodb-file-per-table']

,Parameter,Description
9,innodb-file-per-table,Stores each server table to an .ibd file in th...


In [20]:
# Load BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')


/Users/yoonji_kim/delab/tuning/parsing_mysql_parameter/myvenv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `b

In [21]:
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

In [22]:
# MySQL과 PostgreSQL의 각 Description에 대해 임베딩 계산
embeddings_mysql = [get_bert_embedding(desc) for desc in mysql_csv['Description']]
embeddings_postgresql = [get_bert_embedding(desc) for desc in postgresql_csv['Description']]


In [23]:
mysql_2d = np.array(embeddings_mysql).reshape(np.array(embeddings_mysql).shape[0], np.array(embeddings_mysql).shape[2])
mysql_2d.shape

(45, 768)

In [24]:
psql_2d = np.array(embeddings_postgresql).reshape(np.array(embeddings_postgresql).shape[0], np.array(embeddings_postgresql).shape[2])
psql_2d.shape

(324, 768)

In [25]:
similarities = cosine_similarity(mysql_2d, psql_2d)

In [26]:
similarities.shape

(45, 324)

In [29]:
# 유사도 임계값 설정 (예: 0.8)
threshold = 0.8

# 매칭 리스트 초기화
matches = []

# MySQL 파라미터에 대해 PostgreSQL 파라미터와 매칭
for i in range(len(embeddings_mysql)):
    for j in range(len(embeddings_postgresql)):
        sim_value = similarities[i][j]
        if sim_value >= threshold:
            matches.append((mysql_csv['Parameter'][i], postgresql_csv['Parameter'][j], sim_value))

# 유사도가 높은 순서대로 정렬
matches = sorted(matches, key=lambda x: x[2], reverse=True)


In [33]:
 # CSV 파일로 저장
# with open('mysql_postgresql_matching_knobs_ver1.csv', mode='w', newline='') as file:
with open('../csv/mysql_postgresql_matching_knobs_ver2.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["MySQL Parameter", "PostgreSQL Parameter", "Similarity"])
    for match in matches:
        writer.writerow([match[0], match[1], match[2]])


In [30]:
# 결과 출력
for match in matches:
    print(f"MySQL Parameter: {match[0]}")
    print(f"PostgreSQL Parameter: {match[1]}")
    print(f"Similarity: {match[2]}")
    print("-" * 50)

MySQL Parameter: innodb-buffer-pool-size=#
PostgreSQL Parameter: shared_buffers
Similarity: 0.9186912775039673
--------------------------------------------------
MySQL Parameter: innodb-log-buffer-size=#
PostgreSQL Parameter: shared_buffers
Similarity: 0.917931854724884
--------------------------------------------------
MySQL Parameter: replica-max-allowed-packet=#
PostgreSQL Parameter: wal_receiver_timeout
Similarity: 0.9033223390579224
--------------------------------------------------
MySQL Parameter: join-buffer-size=#
PostgreSQL Parameter: shared_buffers
Similarity: 0.9015588760375977
--------------------------------------------------
MySQL Parameter: max-allowed-packet=#
PostgreSQL Parameter: wal_receiver_timeout
Similarity: 0.8994658589363098
--------------------------------------------------
MySQL Parameter: innodb-buffer-pool-size=#
PostgreSQL Parameter: work_mem
Similarity: 0.8978725671768188
--------------------------------------------------
MySQL Parameter: max-binlog-cache